<a href="https://colab.research.google.com/github/arkapriyathecoderinprogress/PoeticTextGenerator/blob/main/PoeticTextGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#Import necessary libraries
import tensorflow as tf
import numpy as np
import os
import time

In [3]:
# 1. Loading Shakespeare Texts
#Download the Shakespeare dataset
path_to_file=tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

#Read the data
text=open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(f"Length of text: {len(text)} characters")

#Take a look at the first 250 characters in the text
print(text[:250])

1115394/1115394 [==============================] - 0s 0us/step
Length of text: 1115394 characters
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



In [4]:
# 2. Preparing data
#Create a set of unique characters in the text
vocab=sorted(set(text))
print(f"{len(vocab)} unique characters")

#Create a mapping from characters to indices and vice versa
char2idx={u:i for i,u in enumerate(vocab)}
idx2char=np.array(vocab)

#Convert the text to integer representation
text_as_int=np.array([char2idx[c] for c in text])

#Define the maximum length of the sequences
seq_length=100
examples_per_epoch=len(text)//seq_length

#Create training examples/targets
char_dataset=tf.data.Dataset.from_tensor_slices(text_as_int)
sequences=char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
  input_text=chunk[:-1]
  target_text=chunk[:1]
  return input_text,target_text

dataset=sequences.map(split_input_target)

#Batch size and buffer size
BATCH_SIZE=64
BUFFER_SIZE=10000

dataset=dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

65 unique characters


In [10]:
#3. Building Recurrent Neural Network
#Length of vocabulary in chars
vocab_size=len(vocab)

#The embedding dimension
embedding_dim=256

#Number of RNN units
rnn_units=1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model=tf.keras.Sequential([
      tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size,None]),
      tf.keras.layers.GRU(rnn_units,
                          return_sequences=True,
                          stateful=True,
                          recurrent_initializer='glorot_uniform'),
      tf.keras.layers.Dense(vocab_size)
  ])
  return model

model=build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE
)

In [6]:
#Debug: Print dataset shape
for input_example, target_example in dataset.take(1):
    print(f"Input shape: {input_example.shape}")
    print(f"Target shape: {target_example.shape}")

Input shape: (64, 100)
Target shape: (64, 1)


In [7]:
# 4. Helper Function
#Function to compute the loss
def loss(labels,logits):
  return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer='adam', loss=loss)

#Directory where the checkpoints will be saved
checkpoint_dir='./training_checkpoints'
checkpoint_prefix=os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True
)

In [13]:
# 5. Generating Text
#Define the number of epochs
EPOCHS = 20

#Train the model
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

#Function to generate text
def generate_text(model, start_string, temperature=1.0):
    # Number of characters to generate
    num_generate = 1000

    #Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    #Empty string to store our results
    text_generated = []

    #Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        #Using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        #The predicted character is passed as the next input to the model
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return start_string + ''.join(text_generated)

Epoch 1/20
172/172 [==============================] - 965s 6s/step - loss: 2.6317
Epoch 2/20
172/172 [==============================] - 952s 6s/step - loss: 1.9515
Epoch 3/20
172/172 [==============================] - 948s 5s/step - loss: 1.6856
Epoch 4/20
172/172 [==============================] - 958s 6s/step - loss: 1.5394
Epoch 5/20
172/172 [==============================] - 961s 6s/step - loss: 1.4525
Epoch 6/20
172/172 [==============================] - 953s 6s/step - loss: 1.3926
Epoch 7/20
172/172 [==============================] - 955s 6s/step - loss: 1.3474
Epoch 8/20
172/172 [==============================] - 948s 6s/step - loss: 1.3094
Epoch 9/20
172/172 [==============================] - 959s 6s/step - loss: 1.2745
Epoch 10/20
172/172 [==============================] - 944s 5s/step - loss: 1.2418
Epoch 11/20
172/172 [==============================] - 1026s 6s/step - loss: 1.2086
Epoch 12/20
172/172 [==============================] - 958s 6s/step - loss: 1.1765
Epoch 13/20


In [14]:
# 6. Results
#Load the dataset checkpoint
model=build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

#Generate text with low temperature
print("Generated text with low temperature (0.5):\n")
print(generate_text(model, start_string="ROMEO: ", temperature=0.5))

#Generate text with high temperature
print("\nGenerated text with high temperature (1.5):\n")
print(generate_text(model, start_string="ROMEO: ", temperature=1.5))

Generated text with low temperature (0.5):

ROMEO: there is my power,
And let them know the sea that is no wife.

PAULINA:
Good morrow, Catesby,
What corns it then? what can you stial to the sea?

SICINIUS:
Peace!
The devils are out of door!

ANGELO:
That he is to be the thing that will be done!
Thou and thy conscience sake, to see him any here?

Provost:
This is a gentleman to be a prisoner to the king;
For then the sea with hasty Kate,
And then the seat before him, where the death
Was wise and virtuous and woman's leave.

Second Servant:
My lord, I cannot speak a word.

ARCHIDAMUS:
Why, that will find the king at Oxford.

HENRY BOLINGBROKE:
Many and heir and to stand upon my company.

Both:
To bring you so to me, in the duke's death?
O, that you look'd for being the deed to save thee,
I should not be distributed.

Provost:
Peace that hath brought you for a poison, nor witness to
the time, when that is not a drum dead mouth
To play the time to come. This man's bosom first when he did 